In [32]:
from PIL import Image
from torch.utils.data import Dataset
import os
from glob import glob
import torch
import torchvision.transforms as transforms

# Dataset Class for Setting up the data loading process
# Stuff to fill in this script: _init_transform()
class inaturalist(Dataset):
    def __init__(self, root_dir, mode = 'train', transform = True):
        self.data_dir = root_dir
        self.mode = mode
        self.transforms = transform      
        self._init_dataset()
        if transform:
            self._init_transform()

    def _init_dataset(self):
        self.files = []
        self.labels = []
        dirs = sorted(os.listdir(os.path.join(self.data_dir, 'train')))
        if self.mode == 'train': 
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'train', dirs[dir], '*.jpg')))
                self.labels += [dir]*len(files)            
                self.files += files
        elif self.mode == 'val':
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'val', dirs[dir], '*.jpg')))
                self.labels += [dir]*len(files)            
                self.files += files
        else:
            print("No Such Dataset Mode")
            return None
    
    def _init_transform(self):  
        # All images are of different size, so resize is necessary
        self.transform = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                             transforms.ToTensor(), 
                                             transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])])
    
    def __getitem__(self, index):
        img = Image.open(self.files[index]).convert('RGB')
        label = self.labels[index]

        if self.transforms:
            img = self.transform(img)

        label = torch.tensor(label, dtype = torch.long)

        return img, label

    def __len__(self):
        return len(self.files)

In [33]:
train = inaturalist(root_dir='Data/inaturalist_12K', mode='train')
val = inaturalist(root_dir='Data/inaturalist_12K', mode='val')

In [34]:
train[9821][1]

tensor(10)

In [35]:
total_height = 0
total_width = 0
for i in range(9999):
    height = train[i][0].shape[1]
    width = train[i][0].shape[2]
    total_height = total_height + height
    total_width = total_width + width
    
print(total_height/9999, total_width/9999)

total_height = 0
total_width = 0
for i in range(1999):
    height = val[i][0].shape[1]
    width = val[i][0].shape[2]
    total_height = total_height + height
    total_width = total_width + width
    
print(total_height/1999, total_width/1999)

653.6557655765577 739.8230823082308
653.5362681340671 740.7168584292147


In [8]:
min_height_ratio = 1000
min_width_ratio = 1000

for i in range(9999):
    height = train[i][0].shape[1]
    width = train[i][0].shape[2]
    if height/width < min_height_ratio:
        min_height_ratio = height/width
    if width/height < min_width_ratio:
        min_width_ratio = width/height
        
print(min_height_ratio, min_width_ratio)

0.155 0.3825


IndexError: list index out of range

In [9]:
min_height_ratio = 1000
min_width_ratio = 1000

for i in range(1999):
    height = val[i][0].shape[1]
    width = val[i][0].shape[2]
    if height/width < min_height_ratio:
        min_height_ratio = height/width
    if width/height < min_width_ratio:
        min_width_ratio = width/height
        
print(min_height_ratio, min_width_ratio)


0.18375 0.3925


In [38]:
train[1029][0].shape[1]

585

In [39]:
min_height = 1000
min_width = 1000

for i in range(9999):
    height = train[i][0].shape[1]
    width = train[i][0].shape[2]
    if height < min_height:
        min_height = height
    if width < min_width:
        min_width = width
        
print(min_height, min_width)

124 306


In [40]:
val = inaturalist(root_dir='Data/inaturalist_12K', mode='val')

min_height = 1000
min_width = 1000

for i in range(1999):
    height = val[i][0].shape[1]
    width = val[i][0].shape[2]
    if height < min_height:
        min_height = height
    if width < min_width:
        min_width = width
        
print(min_height, min_width)

147 314


In [1]:
from model import Classifier
from torchsummary import summary

model = Classifier(n_classes = 10, layers = [3, 4, 6, 3])

In [3]:
summary(model.cuda(), (3, 299, 299))

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [4]:
import torch.nn as nn
import math

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [5]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 10)

In [6]:
summary(model.cuda(), (3, 512, 512))

RuntimeError: mat1 dim 1 must match mat2 dim 0